In [6]:
import torch

def tensorize(s : str) -> torch.Tensor:
  return torch.Tensor([(1 if (ch >> i) & 1 == 1 else -1) for ch in list(map(ord, s)) for i in range(8)])

class NeuralNetwork(torch.nn.Module):
  def __init__(self, in_dimension, mid_dimension, out_dimension=1):
    super(NeuralNetwork, self).__init__()
    self.fc = torch.nn.Linear(in_dimension, mid_dimension)
    self.fc2 = torch.nn.Linear(mid_dimension, out_dimension)

  def step_activation(self, x : torch.Tensor) -> torch.Tensor:
    x[x <= 0] = -1
    x[x >  0] = 1
    return x

  def forward(self, x : torch.Tensor) -> int:
    x = self.fc(x)
    x = self.step_activation(x)
    x = self.fc2(x)
    x = self.step_activation(x)
    return int(x)

flag = input("Enter flag: ")
in_data = tensorize(flag)
in_dim	= len(in_data)

model = NeuralNetwork(in_dim, 1024)
model.load_state_dict(torch.load("chal.pth"))

if model(in_data) == 1:
	print("Yay you did it Good Job!!!")
else:
	print("Yay you did it Good Job!!!")

RuntimeError: Error(s) in loading state_dict for NeuralNetwork:
	size mismatch for fc.weight: copying a param with shape torch.Size([1024, 128]) from checkpoint, the shape in current model is torch.Size([1024, 40]).

In [19]:
import torch

def tensorize(s: str) -> torch.Tensor:
    return torch.Tensor([(1 if (ch >> i) & 1 == 1 else -1) for ch in list(map(ord, s)) for i in range(8)])

checkpoint = torch.load("chal.pth", map_location=torch.device('cpu'))

# Inspect the keys of the checkpoint to understand its structure
print(checkpoint.keys())

# Now let's inspect the weights and biases of the linear layers
print("First Linear Layer Weights:", checkpoint['fc.weight'].shape)
print("First Linear Layer Biases:", checkpoint['fc.bias'].shape)
print("Second Linear Layer Weights:", checkpoint['fc2.weight'].shape)
print("Second Linear Layer Biases:", checkpoint['fc2.bias'].shape)

# Based on the shape of the weights and biases, try to infer how the model is making decisions

# Assuming it's a binary classification problem, craft an input that would result in the desired output
# You may need to experiment with different inputs based on your analysis of the weights and biases
# For instance, you can try to find an input that would result in the final output being close to 1

# Once you have a candidate input, you can convert it to the string representation to get the flag
# You may need to reverse the `tensorize` function to convert the tensor back to a string

# flag = reverse_tensorize(candidate_input_tensor)
# print("Flag:", flag)


odict_keys(['fc.weight', 'fc.bias', 'fc2.weight', 'fc2.bias'])
First Linear Layer Weights: torch.Size([1024, 128])
First Linear Layer Biases: torch.Size([1024])
Second Linear Layer Weights: torch.Size([1, 1024])
Second Linear Layer Biases: torch.Size([1])


In [73]:
import torch

def tensorize(s: str) -> torch.Tensor:
    return torch.Tensor([(1 if (ch >> i) & 1 == 1 else -1) for ch in list(map(ord, s)) for i in range(8)])

checkpoint = torch.load("chal.pth", map_location=torch.device('cpu'))

# Define the model architecture based on the loaded checkpoint
fc_weight = checkpoint['fc.weight']
fc_bias = checkpoint['fc.bias']
fc2_weight = checkpoint['fc2.weight']
fc2_bias = checkpoint['fc2.bias']

# Define a function to make predictions based on the model parameters
def predict(input_data):
    x = torch.matmul(fc_weight, input_data) + fc_bias
    x = torch.relu(x)  # Assuming ReLU activation for the first layer
    x = torch.matmul(fc2_weight, x) + fc2_bias
    return torch.sigmoid(x)  # Assuming sigmoid activation for the second layer

# Generate random inputs and check if any produce an output close to 1
for _ in range(1000):  # Try 1000 random inputs
    random_input = torch.randn((128, 1))  # Generate a random input of size 128
    output = predict(random_input)
    if (output == 1):  # If any element of the output is close to 1
        flag_tensor = random_input
        break

# Convert the tensor back to a string representation to get the flag
flag = ""
for i in range(0, len(flag_tensor), 8):
    byte = flag_tensor[i:i+8]
    char_code = int("".join('1' if bit.item() > 0 else '0' for bit in byte), 2)
    flag += chr(char_code)

print("Flag:", flag)


RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [78]:
import torch
import itertools

def tensorize(s: str) -> torch.Tensor:
    return torch.Tensor([(1 if (ch >> i) & 1 == 1 else -1) for ch in list(map(ord, s)) for i in range(8)])

checkpoint = torch.load("chal.pth", map_location=torch.device('cpu'))

# Define the model architecture based on the loaded checkpoint
fc_weight = checkpoint['fc.weight']
fc_bias = checkpoint['fc.bias']
fc2_weight = checkpoint['fc2.weight']
fc2_bias = checkpoint['fc2.bias']

# Define a function to make predictions based on the model parameters
def predict(input_data):
    x = torch.matmul(fc_weight, input_data.float()) + fc_bias.float()  # Ensure both tensors are of the same dtype
    x = torch.relu(x)  # Assuming ReLU activation for the first layer
    x = torch.matmul(fc2_weight, x) + fc2_bias  # No need to convert this since fc2_bias is already float type
    return torch.sigmoid(x)  # Assuming sigmoid activation for the second layer

# Generate all possible combinations of flags
for bits in itertools.product([-1, 1], repeat=128):  # Generate all possible combinations of 128 bits
    flag_tensor = torch.tensor(bits).unsqueeze(1)
    output = predict(flag_tensor)
    if output == 1:  # If output is exactly 1
        correct_flag = "".join('1' if bit.item() > 0 else '0' for bit in flag_tensor)
        print("Correct Flag:", correct_flag)
        break


RuntimeError: Boolean value of Tensor with more than one value is ambiguous